In [1]:
import os
import tensorflow as tf
from util import constants
from util.config_util import get_model_params, get_task_params, get_train_params
from tf2_models.trainer import Trainer
from absl import app
from absl import flags
import numpy as np
from util.models import MODELS
from util.tasks import TASKS
from notebook_utils import *

%matplotlib inline
import pandas as pd
import seaborn as sns; sns.set()

from tqdm import tqdm

In [ ]:
student_exp_name='lisa_fd7'
teacher_exp_name='0.001_lisa_offlineteacher_v1'
teacher_config='lstm_drop31_v2'
task_name = 'word_sv_agreement_lm'
student_model='lm_gpt2'
teacher_model='lm_lstm_shared_emb'
student_config='very_big_gpt_v10'
distill_config='pure_distill_4'
distill_mode='offline'

chkpt_dir='../tf_ckpts'

In [ ]:
student_exp_name='lisa_fd10'
teacher_exp_name='0.001_lisa_offlineteacher_v1'
teacher_config='lstm_drop31_v2'
task_name = 'word_sv_agreement_lm'
student_model='lm_lstm_shared_emb'
teacher_model='lm_lstm_shared_emb'
student_config='lstm_drop31_v2'
distill_config='pure_distill_4'
distill_mode='offline'

chkpt_dir='../tf_ckpts'

In [ ]:
student_exp_name='lisa_fd8'
teacher_exp_name='0.0001_lisa_offlineteacher_v1'
teacher_config='very_big_gpt_v10'
task_name = 'word_sv_agreement_lm'
student_model='lm_gpt2'
teacher_model='lm_gpt2'
student_config='very_big_gpt_v10'
distill_config='pure_distill_4'
distill_mode='offline'

chkpt_dir='../tf_ckpts'

In [ ]:
student_exp_name='lisa_fd9'
teacher_exp_name='0.0001_lisa_offlineteacher_v1'
teacher_config='very_big_gpt_v10'
task_name = 'word_sv_agreement_lm'
student_model='lm_lstm_shared_emb'
teacher_model='lm_gpt2'
student_config= 'lstm_drop31_v2'
distill_config='pure_distill_4'
distill_mode='offline'



In [2]:
task_name = 'word_sv_agreement_lm'
chkpt_dir='../tf_ckpts'
task = TASKS[task_name](get_task_params(), data_dir='../data')
cl_token = task.databuilder.sentence_encoder().encode(constants.bos)


Vocab len:  10032


In [ ]:
teacher_model = MODELS[teacher_model](hparams=get_model_params(task, teacher_model, teacher_config), cl_token=cl_token)
std_hparams=get_model_params(task, student_model, student_config)
std_hparams.output_attentions = True
std_hparams.output_embeddings = True
student_model = MODELS[student_model](
std_hparams, cl_token=cl_token)

In [ ]:
student_ckpt_dir = os.path.join(chkpt_dir, task.name,
                              '_'.join([distill_mode,distill_config,
                                        "teacher", teacher_model.model_name, 
                                        teacher_config,
                                        teacher_exp_name,
                                       "student",student_model.model_name,
                                        str(student_config),
                                        student_exp_name]))
print("student_checkpoint:", student_ckpt_dir)

student_ckpt = tf.train.Checkpoint(net=student_model)
student_manager = tf.train.CheckpointManager(student_ckpt, student_ckpt_dir, max_to_keep=None)

student_ckpt.restore(student_manager.latest_checkpoint)
if student_manager.latest_checkpoint:
  print("Restored student from {}".format(student_manager.latest_checkpoint))

student_model.compile(loss=task.get_loss_fn(), metrics=task.metrics())

In [ ]:
teacher_ckpt_dir = os.path.join(chkpt_dir, task.name,
                                  '_'.join([teacher_model.model_name, teacher_config,teacher_exp_name]))

teacher_ckpt = tf.train.Checkpoint(net=teacher_model)
teacher_manager = tf.train.CheckpointManager(teacher_ckpt, teacher_ckpt_dir, max_to_keep=None)

teacher_ckpt.restore(teacher_manager.latest_checkpoint)
if teacher_manager.latest_checkpoint:
  print("Restored student from {}".format(teacher_manager.latest_checkpoint))

teacher_model.compile(loss=task.get_loss_fn(), metrics=task.metrics())

In [ ]:
train = teacher_model.evaluate(task.train_dataset, steps=task.n_train_batches)
valid = teacher_model.evaluate(task.valid_dataset, steps=task.n_valid_batches)
test = teacher_model.evaluate(task.test_dataset, steps=task.n_test_batches)

print("train:", train)
print("valid:", valid)
print("test:", test)

In [ ]:
train = student_model.evaluate(task.train_dataset, steps=task.n_train_batches)
valid = student_model.evaluate(task.valid_dataset, steps=task.n_valid_batches)
test = student_model.evaluate(task.test_dataset, steps=task.n_test_batches)

print("train:", train)
print("valid:", valid)
print("test:", test)

In [3]:
config = {'model_name':'lm_lstm_shared_emb',
            'model_config':'lstm_drop31_v2',
            'learning_rate':0.001,
            'exp_name':'lisa_crs_fst_offlineteacher_v23',
            'chkpt_dir': '../tf_ckpts'
    }
hparams=get_model_params(task, config['model_name'], config['model_config'])
hparams.output_attentions = True
hparams.output_embeddings = True

model, ckpt = get_model(config, task, hparams, cl_token)

train = model.evaluate(task.train_dataset, steps=task.n_train_batches)
valid = model.evaluate(task.valid_dataset, steps=task.n_valid_batches)
test = model.evaluate(task.test_dataset, steps=task.n_test_batches)

print("train:", train)
print("valid:", valid)
print("test:", test)

model config: lstm_drop31_v2
{'hidden_dim': 512, 'embedding_dim': 512, 'depth': 2, 'hidden_dropout_rate': 0.3, 'input_dropout_rate': 0.2}
Restored student from ../tf_ckpts/word_sv_agreement_lm/lm_lstm_shared_emb_em-512_h-512_d-2_hdrop-0.3_indrop-0.2_lstm_drop31_v2_0.001_lisa_crs_fst_offlineteacher_v23/ckpt-60
100/100 [==============================] - 8s 79ms/step - loss: 4.0966 - batch_masked_sequence_loss: 4.0966 - masked_batch_perplexity: 60.3868 - masked_perplexity: 78.1307 - accuracy: 0.0730 - accuracy_top2: 0.0938 - accuracy_top5: 0.1203


[4.096584298610687,
 4.0965834,
 60.386765,
 78.13074,
 0.07295053,
 0.093817,
 0.120311886]

In [ ]:
model_name='lm_gpt2'
model_config='big_gpt_v5'
learning_rate=0.0001
exp_name='lisa_offlineteacher_v3'
task_name = 'word_sv_agreement_lm'
chkpt_dir = '../tf_ckpts'

task = TASKS[task_name](get_task_params(), data_dir='../data')

cl_token = task.databuilder.sentence_encoder().encode(constants.bos)
hparams=get_model_params(task, model_name, model_config)
hparams.output_attentions = True
hparams.output_embeddings = True

model = MODELS[model_name](hparams=hparams, cl_token=cl_token)


ckpt_dir = os.path.join(chkpt_dir,task.name,
                        model.model_name+"_"+str(model_config)+"_"+str(learning_rate)+"_"+exp_name)

ckpt = tf.train.Checkpoint(net=model)
manager = tf.train.CheckpointManager(ckpt, ckpt_dir, max_to_keep=None)

ckpt.restore(manager.latest_checkpoint)
if manager.latest_checkpoint:
  print("Restored student from {}".format(manager.latest_checkpoint))
else:
 print("Initialized from scratch")
 print(ckpt_dir)

model.compile(loss=task.get_loss_fn(), metrics=task.metrics())



train = model.evaluate(task.train_dataset, steps=task.n_train_batches)
print("train:", train)

valid = model.evaluate(task.valid_dataset, steps=task.n_valid_batches)
print("valid:", valid)

test = model.evaluate(task.test_dataset, steps=task.n_test_batches)
print("test:", test)